<div align=center><img src="./assets/cover.png" alt="bert-cover"></div>

<div align=center><img src="./assets/contents_1_2.png" alt="table-of-contents"></div>

# NLP中的预训练模型

语言模型的演变经历了以下几个阶段：

<div align=center><img src="./assets/language_models.jpg" alt="language-models" width="1300"></div>

1. `word2vec`/`Glove`将离散的文本数据转换为固定长度的**静态词向量**，后根据下游任务训练不同的**语言模型**；

2. `ELMo`预训练模型将文本数据*结合上下文信息*，转换为**动态词向量**，后根据下游任务训练不同的**语言模型**；

3. `BERT`同样将文本数据转换为**动态词向量**，能够更好地捕捉*句子级别的信息与语境信息*，后续只需finetune最后的**输出层**即可适配下游任务；

4. `GPT`等预训练语言模型主要用于*文本生成类任务*，需要通过**prompt方法**来应用于下游任务，指导模型生成特定的输出。

# BERT

<div align=center><img src="./assets/BERT.jpg" width="800"></div>

BERT模型本质上是结合了`ELMo`模型与`GPT`模型的优势。

- 相比于ELMo，BERT仅需改动最后的输出层，而非模型架构，便可以在下游任务中达到很好的效果；
- 相比于GPT，BERT在处理词元表示时考虑到了双向上下文的信息；


## BERT的微调思路

<div align=center><img src="./assets/bert_overview.png" alt="bert-overview" width="1000"></div>

BERT作为预训练模型，用来处理一般的语言理解任务，其中下游任务主要分为以下四大类：
1. 单句子分类任务（如情感分析）；
2. 句子对分类任务（如判断输入的两个句子在语义上是否等效）；
3. 问答任务（给定描述，找到描述中针对问题的答案）；
4. 文本标注任务（如命名体识别）；

# BERT 结构

BERT（Bidirectional Encoder Representation from Transformers）是一个**仅有Encoder的Transformer**。

我们调用`BertModel`来搭建BERT模型，并通过`BertConfig`配置模型相关参数。

In [ ]:
from pretrain.src.bert import BertModel
from pretrain.src.config import BertConfig

## BERT 输入

- `Transformer`：Encoder接受源句子（source sentence），Decoder接受目标句子（target sentence）；
- `BERT`: 
    - 针对句子对相关任务，将两个句子合并为一个句子对输入到Encoder中，\<cls\> + 第一个句子 + \<sep\> + 第二个句子 + \<sep\>;
    - 针对单个文本相关任务，\<cls\> + 句子 + \<sep\>。

<div align=center><img src="./assets/transformer_vs_bert.png" alt="transformer-vs-bert"></div>

## BERT Embedding

BERT Embedding与Transformer中的Embedding操作类似，包含词嵌入（word embedding）与位置嵌入（positional embedding）。

但与Transformer不同的是，BERT使用了可学习的位置信息，并额外增加了表示区分不同句子的段嵌入（segment embedding）。

<div align=center><img src="./assets/BERT-embedding.ppm" alt="bert-embedding" width="1500"></div>

> <font size=2>图片来源：Devlin, J.; Chang, M. W.; Lee, K.; Toutanova, K. BERT: Pre-training of deep bidirectional transformers for language understanding. arXiv preprint arXiv:1810.04805, 2018.</font>

In [ ]:
import mindspore
from mindspore import nn
import mindspore.common.dtype as mstype
from mindspore.common.initializer import initializer, TruncatedNormal

class BertEmbeddings(nn.Cell):
    """
    Embeddings for BERT, include word, position and token_type
    """
    def __init__(self, config):
        super().__init__()
        self.word_embeddings = nn.Embedding(config.vocab_size, config.hidden_size, embedding_table=TruncatedNormal(config.initializer_range))
        self.position_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size, embedding_table=TruncatedNormal(config.initializer_range))
        self.token_type_embeddings = nn.Embedding(config.type_vocab_size, config.hidden_size, embedding_table=TruncatedNormal(config.initializer_range))
        self.layer_norm = nn.LayerNorm((config.hidden_size,), epsilon=config.layer_norm_eps)
        self.dropout = nn.Dropout(1 - config.hidden_dropout_prob)

    def construct(self, input_ids, token_type_ids=None, position_ids=None):
        seq_len = input_ids.shape[1]
        if position_ids is None:
            position_ids = mnp.arange(seq_len)
            position_ids = position_ids.expand_dims(0).expand_as(input_ids)
        if token_type_ids is None:
            token_type_ids = ops.zeros_like(input_ids)
        
        words_embeddings = self.word_embeddings(input_ids)
        position_embeddings = self.position_embeddings(position_ids)
        token_type_embeddings = self.token_type_embeddings(token_type_ids)
        embeddings = words_embeddings + position_embeddings + token_type_embeddings
        embeddings = self.layer_norm(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings


## BERT 模型构建

BERT模型的构建与上一节课程的Transformer Encoder构建类似。

分别构建multi-head attention层，feed-forward network，并在中间用add&norm连接，最后通过线性层与softmax层进行输出。

## BERT 预训练

BERT通过Masked LM（masked language model）与NSP（next sentence prediction）获取词语和句子级别的特征。

<div align=center><img src="./assets/bert_pretrain_finetune.jpg" alt="bert-pretrain"></div>

> <font size=2>图片来源：Devlin, J.; Chang, M. W.; Lee, K.; Toutanova, K. BERT: Pre-training of deep bidirectional transformers for language understanding. arXiv preprint arXiv:1810.04805, 2018.</font>

### Masked Language Model (Masked LM)

BERT模型通过Masked LM捕捉**词语层面**的信息。

我们随机将每个句子中15%的词语进行遮盖，替换成掩码\<mask\>。在训练过程中，模型会对句子进行“完形填空”，预测这些被遮盖的词语是什么，通过减小被mask词语的损失值来对模型进行优化。

<div align=center><img src="./assets/masked_lm.png" alt="masked-lm"></div>

> <font size=2>图片来源: [The Illustrated BERT, ELMo, and co. (How NLP Cracked Transfer Learning)](https://jalammar.github.io/illustrated-bert)</font>

由于\<mask\>仅在预训练中出现，为了让预训练和微调中的数据处理尽可能接近，我们在随机mask的时候进行如下操作：
- 80%的概率替换为\<mask\>
- 10%的概率替换为文本中的随机词
- 10%的概率不进行替换，保持原有的词元

<div align=center><img src="./assets/masked_lm_2.png" alt="masked-lm-2"></div>

我们通过`BERTPredictionHeadTranform`实现单层感知机，对被遮盖的词元进行预测。在前向网络中，我们需要输入BERT模型的编码结果`hidden_states`。

In [ ]:
activation_map = {
    'relu': nn.ReLU(),
    'gelu': nn.GELU(False),
    'gelu_approximate': nn.GELU(),
    'swish':nn.SiLU()
}

class BertPredictionHeadTransform(nn.Cell):
    def __init__(self, config):
        super().__init__()
        self.dense = nn.Dense(config.hidden_size, config.hidden_size, weight_init=TruncatedNormal(config.initializer_range))
        self.transform_act_fn = activation_map.get(config.hidden_act, nn.GELU(False))
        self.layer_norm = nn.LayerNorm((config.hidden_size,), epsilon=config.layer_norm_eps)
    
    def construct(self, hidden_states):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.transform_act_fn(hidden_states)
        hidden_states = self.layer_norm(hidden_states)
        return hidden_states

根据被遮盖的词元位置`masked_lm_positions`，获得这些词元的预测输出。

In [ ]:
import mindspore.ops as ops
import mindspore.numpy as mnp
from mindspore import Parameter, Tensor

class BertLMPredictionHead(nn.Cell):
    def __init__(self, config):
        super(BertLMPredictionHead, self).__init__()
        self.transform = BertPredictionHeadTransform(config)

        # The output weights are the same as the input embeddings, but there is
        # an output-only bias for each token.
        self.decoder = nn.Dense(config.hidden_size, config.vocab_size, has_bias=False, weight_init=TruncatedNormal(config.initializer_range))

        self.bias = Parameter(initializer('zeros', config.vocab_size), 'bias')

    def construct(self, hidden_states, masked_lm_positions):
        # hidden_states: [batch_size, seq_len, hidden_size]
        batch_size, seq_len, hidden_size = hidden_states.shape
        if masked_lm_positions is not None:
            # flat_offfsets: [batch_size, ]
            flat_offsets = mnp.arange(batch_size) * seq_len
            # flat_position: [batch_size * mask_len, ]
            flat_position = (masked_lm_positions + flat_offsets.reshape(-1, 1)).reshape(-1)
            # flat_sequence_tensor: [batch_size * seq_len, hidden_size]
            flat_sequence_tensor = hidden_states.reshape(-1, hidden_size)
            # hidden_states: [batch_size * mask_len, hidden_size]
            hidden_states = ops.gather(flat_sequence_tensor, flat_position, 0)
        hidden_states = self.transform(hidden_states)
        hidden_states = self.decoder(hidden_states) + self.bias
        return hidden_states

### Next Sentence Prediction (NSP)

BERT通过NSP捕捉**句子级别**的信息，使其可以理解句子与句子之间的联系，从而能够应用于问答或者推理任务。

NSP本质上是一个**二分类任务**，通过输入一个句子对，判断两句话是否为连续句子。输入的两个句子A和B中，B有50%的概率是A的下一句。

<div align=center><img src="./assets/nsp.png" alt="nsp"></div>

> <font size=2>图片来源: [The Illustrated BERT, ELMo, and co. (How NLP Cracked Transfer Learning)](https://jalammar.github.io/illustrated-bert)</font>

另外，输入的内容最好是document-level的语料，而非sentence-level的语料，这样训练出的模型可以具备抓取长序列特征的能力。

在这里，我们使用一个单隐藏层的多层感知机`BERTPooler`进行二分类预测。因为特殊占位符\<cls\>在预训练中对应了句子级别的特征信息，所以多层感知机分类器只需要输出\<cls\>对应的隐藏层输出。

In [ ]:
class BertPooler(nn.Cell):
    def __init__(self, config):
        super(BertPooler, self).__init__()
        self.dense = nn.Dense(config.hidden_size, config.hidden_size, activation='tanh', weight_init=TruncatedNormal(config.initializer_range))
    
    def construct(self, hidden_states):
        # We "pool" the model by simply taking the hidden state corresponding.
        # to the first token
        first_token_tensor = hidden_states[:, 0]
        pooled_output = self.dense(first_token_tensor)
        return pooled_output

最后，多层感知机分类器的输出通过一个线性层`self.seq_relationship`，输出对nsp的预测。

在`BERTPreTrainingHeads`中，我们对以上提到的两种方式进行整合。最终输出Maked LM（`prediction scores`）和NSP（`seq_realtionship_score`）的预测结果。

In [ ]:
class BertPreTrainingHeads(nn.Cell):
    def __init__(self, config):
        super(BertPreTrainingHeads, self).__init__()
        self.predictions = BertLMPredictionHead(config)
        self.seq_relationship = nn.Dense(config.hidden_size, 2, weight_init=TruncatedNormal(config.initializer_range))
    
    def construct(self, sequence_output, pooled_output, masked_lm_positions):
        prediction_scores = self.predictions(sequence_output, masked_lm_positions)
        seq_relationship_score = self.seq_relationship(pooled_output)
        return prediction_scores, seq_relationship_score

### BERT预训练代码整合

我们将上述的类进行实例化，并借此回顾一下BERT预训练的整体流程。

1. `BertModel`构建BERT模型；
2. `BertPretrainingHeads`整合了Masked LM与NSP两个训练任务， 输出预测结果；
    - `BertLMPredictionHead`：输入BERT编码与\<mask\>的位置，输出对应位置词元的预测；
    - `BERTPooler`：输入BERT编码，输出对\<cls\>的隐藏状态，并在`BertPretrainingHeads`中通过线性层输出预测结果；

In [ ]:
class BertForPretraining(nn.Cell):
    def __init__(self, config, *args, **kwargs):
        super().__init__(config, *args, **kwargs)
        self.bert = BertModel(config)
        self.cls = BertPreTrainingHeads(config)
        self.vocab_size = config.vocab_size

        self.cls.predictions.decoder.weight = self.bert.embeddings.word_embeddings.embedding_table

    def construct(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None, masked_lm_positions=None):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask
        )
        # ic(outputs) # [shape(batch_size, 128, 256), shape(batch_size, 256)]

        sequence_output, pooled_output = outputs[:2]
        prediction_scores, seq_relationship_score = self.cls(sequence_output, pooled_output, masked_lm_positions)

        outputs = (prediction_scores, seq_relationship_score,) + outputs[2:]
        # ic(outputs) # [shape(batch_size, 128, 256), shape(batch_size, 256)]

        return outputs